In [1]:
!pip install langchain streamlit -q

In [2]:
# from langchain_community.chat_models import ChatOllama
import streamlit as st # to render the user interface.
from langchain_community.llms import Ollama # to use Ollama llms in langchain
from langchain_core.prompts import ChatPromptTemplate # crafts prompts for our llm
from langchain_community.chat_message_histories import\
StreamlitChatMessageHistory # stores message history
from langchain_core.tools import tool # tools for our llm
from langchain.tools.render import render_text_description # to describe tools as a string 
from langchain_core.output_parsers import JsonOutputParser # ensure JSON input for tools
from operator import itemgetter # to retrieve specific items in our chain.

In [4]:
# Set up the LLM which will power our application.
model = Ollama(model='llama3')

In [5]:
@tool
def add(first: int, second: int) -> int:
    "Add two integers."
    return first + second

In [6]:
print(add.name)
print(add.description)
print(add.args)

add
add(first: int, second: int) -> int - Add two integers.
{'first': {'title': 'First', 'type': 'integer'}, 'second': {'title': 'Second', 'type': 'integer'}}


In [7]:
add.invoke({'first':3, 'second':6})

9

In [8]:
@tool
def multiply(first: int, second: int) -> int:
    """Multiply two integers together."""
    return first * second

In [9]:
@tool
def converse(input: str) -> str:
    "Provide a natural language response using the user input."
    return model.invoke(input)

In [10]:
tools = [add, multiply, converse]
rendered_tools = render_text_description(tools)
print(rendered_tools)

add: add(first: int, second: int) -> int - Add two integers.
multiply: multiply(first: int, second: int) -> int - Multiply two integers together.
converse: converse(input: str) -> str - Provide a natural language response using the user input.


In [11]:
system_prompt = f"""You are an assistant that has access to the following set of tools.
Here are the names and descriptions for each tool:

{rendered_tools}
Given the user input, return the name and input of the tool to use.
Return your response as a JSON blob with 'name' and 'arguments' keys.
The value associated with the 'arguments' key should be a dictionary of parameters."""

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [13]:
chain = prompt | model | JsonOutputParser()

In [14]:
chain.invoke({'input': 'What is 3 times 23'})

{'name': 'multiply', 'arguments': {'first': 3, 'second': 23}}

In [15]:
chain.invoke({'input': 'How are you today?'})

{'tool': 'converse',
 'arguments': {'input': "I'm doing well, thanks for asking. How about you?"}}

In [16]:
# Define a function which returns the chosen tool
# to be run as part of the chain.
def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

In [17]:
chain = prompt | model | JsonOutputParser() | tool_chain
chain.invoke({'input': 'What is 3 times 23'})

69

In [22]:
if 'key' not in st.session_state:
    st.session_state['key'] = 'langchain_messages'

In [23]:
# Set up message history.
msgs = StreamlitChatMessageHistory(key="langchain_messages")
if len(msgs.messages) == 0:
    msgs.add_ai_message("I can add, multiply, or just chat! How can I help you?")

KeyError: 'st.session_state has no key "langchain_messages". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization'